Analise de Sentimento realtime no Twitter

** Configurações utilizadas **

Vamos utilizar Framework Spark para processar dados do Twitter e utilizar API do próprio twitter para captura dos dados

In [0]:
!pip install tweepy
!pip install textblob

  Using cached tweepy-4.13.0-py3-none-any.whl (102 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Not uninstalling requests at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-bd721bcb-4882-41fc-aa2f-b5436079d8cf
    Can't uninstall 'requests'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-bd721bcb-4882-41fc-aa2f-b5436079d8cf/bin/python -m pip install --upgrade pip' command.
  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached regex-2022.10.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (769 kB)
You

In [0]:
# Modulos usados
import numpy as np
import pandas as pd
import tweepy as tw
import matplotlib as plt
import seaborn as sns
import textblob
from IPython.display import display
from textblob import TextBlob
import re
from time import gmtime, strftime
import time


In [0]:
# Autenticação do Twitter, necessario gerar a alteticação no modo conta de desenvolvedor
#colocar dentro das aspas simples os seus dados de acesso a API do Twitter

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOZokAEAAAAADtImcKxLMe2sDjSAgirPqIBks3w%3DJKCYflJMbPuMJ0jotMhaO2D3xwuCtQfJrkHcWr6Vd6ULQ6ILol'
consumer_key = '22KAGbI41T5QkZexzHUfm0Sey'
consumer_secret = 'pNHQvAOoKPTCC9wFMgoovymSSUZ2UuYxW0JolypJ0QRNvRwIm8'
access_token = '1564305191163379712-kpF5EKB1AUo4RevoFriRVa4SIBb2gm'
access_token_secret = 'pExGcHn7xXmJMIlwwr9JSveN66vbAA4oAokjyucUltu9U'

In [0]:
cliente = tw.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret)

In [0]:
search_tweet = cliente.search_recent_tweets(
    "#SANTANDER -is:retweet lang:pt",
      max_results = 100
#     tweet_fields = ['author_id','created_at','text','source','lang'],
#     user_fields = ['name','username','location','verified'],
#     place_fields = ['country','country_code']
)

for tweet in search_tweet.data:
    #print(tweet.author_id)      # print the author id of the tweet
    print(tweet.text)           # print the text
    #print(tweet.data['lang'])   # print the language (PL, since we're filtering by it)
   # print(tweet.data['source']) # what did the user use to publish the tweet?

Sou cliente do porto seguro . Nubank  , BRB , digo Bradesco . Btg .
E tive mais .
O @santander_br  foi o melhor para mim e estou dando prioridade à ele , isso é reciprocidade.
#Santander 😊✌️
#unique #mastercardblack 
40 mil pre aprovado  , financiar  carro .
Precisa aumentar 👌
@santander_br Meu relacionamento com banco é recíproco. Estamos nos entendendo muito bem . As duas partes , estão fazendo as suas partes e estou feliz por isso .
Obrigado #Santander 😊✌️
@santander_br Estou aproveitando @santander_br 
Gostei muito do meu cartão #mastercardblack #unique #Santander 
Limite maravilhoso 😊✌️
Seu apoio é importante com click no banner...
Ou segue 2JN ou fica desinformado Compartilhaê!!! 
#2JN #assaltoabancoemperiperi #santander #periperi
Homens armados explodem agência bancária e provocam pânico em moradores do subúrbio de Salvador https://t.co/EsSsFJ161m
@JdelArco33 decide no 59:59 para seguir facendo ✨HISTORIA✨

😏 Segundo ano consecutivo na F8 da #CopadelRey 🏆 🥳 

🔜 #Santander

#AtaOF

In [0]:
search_tweet

Out[41]: Response(data=[<Tweet id=1636333961356910593 text='Sou cliente do porto seguro . Nubank  , BRB , digo Bradesco . Btg .\nE tive mais .\nO @santander_br  foi o melhor para mim e estou dando prioridade à ele , isso é reciprocidade.\n#Santander 😊✌️\n#unique #mastercardblack \n40 mil pre aprovado  , financiar  carro .\nPrecisa aumentar 👌'>, <Tweet id=1636331315703209986 text='@santander_br Meu relacionamento com banco é recíproco. Estamos nos entendendo muito bem . As duas partes , estão fazendo as suas partes e estou feliz por isso .\nObrigado #Santander 😊✌️'>, <Tweet id=1636326945909612544 text='@santander_br Estou aproveitando @santander_br \nGostei muito do meu cartão #mastercardblack #unique #Santander \nLimite maravilhoso 😊✌️'>, <Tweet id=1636315990739877890 text='Seu apoio é importante com click no banner...\nOu segue 2JN ou fica desinformado Compartilhaê!!! \n#2JN #assaltoabancoemperiperi #santander #periperi\nHomens armados explodem agência bancária e provocam pânico em mo

In [0]:
# Imprimindo Tweets
tweets = search_tweet.data
base = []

for i in tweets:        
    texto = i.text
    
#     if('RT' in texto):
#         posicao = texto.find(':')
#         texto = texto[posicao+2:]
    dados = texto.lower()
    base.append(dados)

# Agrupando os tweets em um dataframe
df = pd.DataFrame(base,columns=['texto_twitter'])
pd.options.display.max_colwidth = 130 
df.head()

,texto_twitter
0,"sou cliente do porto seguro . nubank , brb , digo bradesco . btg .\ne tive mais .\no @santander_br foi o melhor para mim e e..."
1,"@santander_br meu relacionamento com banco é recíproco. estamos nos entendendo muito bem . as duas partes , estão fazendo as s..."
2,@santander_br estou aproveitando @santander_br \ngostei muito do meu cartão #mastercardblack #unique #santander \nlimite marav...
3,seu apoio é importante com click no banner...\nou segue 2jn ou fica desinformado compartilhaê!!! \n#2jn #assaltoabancoemperipe...
4,@jdelarco33 decide no 59:59 para seguir facendo ✨historia✨\n\n😏 segundo ano consecutivo na f8 da #copadelrey 🏆 🥳 \n\n🔜 #santan...


In [0]:
data = df
data['texto_twitter'] = np.array([tweet.text for tweet in tweets])
#Retirando os duplicados
#df.drop_duplicates(['texto_twitter'], inplace=True)

In [0]:
data

,texto_twitter
0,"Sou cliente do porto seguro . Nubank , BRB , digo Bradesco . Btg .\nE tive mais .\nO @santander_br foi o melhor para mim e e..."
1,"@santander_br Meu relacionamento com banco é recíproco. Estamos nos entendendo muito bem . As duas partes , estão fazendo as s..."
2,@santander_br Estou aproveitando @santander_br \nGostei muito do meu cartão #mastercardblack #unique #Santander \nLimite marav...
3,Seu apoio é importante com click no banner...\nOu segue 2JN ou fica desinformado Compartilhaê!!! \n#2JN #assaltoabancoemperipe...
4,@JdelArco33 decide no 59:59 para seguir facendo ✨HISTORIA✨\n\n😏 Segundo ano consecutivo na F8 da #CopadelRey 🏆 🥳 \n\n🔜 #Santan...
5,🔴 GPT-4 de OpenAI - Primeras impresiones... 🔥 https://t.co/QEoIaw4pPA #PayPay配布 #PayPal #zelle #bizumods #Venmo #cashapp #bbva...
6,#Repost @gina.bancaria with @let.repost \n• • • • • •\nBora trabaia\n.\n.\n.\n.\n\n.\n.\n\n#bancario \n#bancaria \n#febraban \...
7,Uma das agências do Santander vai fechar no dia 15 de junho e sua localização pode surpreender. Confira o comunicado do banco!...
8,Preparado pra prova e para as outras fases que vierem. Vamo dale!\n#cnb #streamerup #streamerbr #twitchbr #gamerbr #cnbacademy...
9,🏦 #Santander (SANB11) sobe juros do crédito imobiliário pela segunda vez no ano\n#mercadofinanceiro #investimentos\nhttps://t....


In [0]:
 def clean_tweet(df):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|       (\w+:\/\/\S+)", " ", df).split())


In [0]:
def analize_sentiment(df):
    analysis = TextBlob(clean_tweet(df))
    if analysis.sentiment.polarity > 0:
       return 1
    elif analysis.sentiment.polarity == 0:
       return 0
    else:
       return -1

In [0]:
data['SA'] = np.array([ analize_sentiment(tweet) for tweet in data['texto_twitter'] ])

In [0]:
data

,texto_twitter,SA
0,"Sou cliente do porto seguro . Nubank , BRB , digo Bradesco . Btg .\nE tive mais .\nO @santander_br foi o melhor para mim e e...",1
1,"@santander_br Meu relacionamento com banco é recíproco. Estamos nos entendendo muito bem . As duas partes , estão fazendo as s...",0
2,@santander_br Estou aproveitando @santander_br \nGostei muito do meu cartão #mastercardblack #unique #Santander \nLimite marav...,1
3,Seu apoio é importante com click no banner...\nOu segue 2JN ou fica desinformado Compartilhaê!!! \n#2JN #assaltoabancoemperipe...,0
4,@JdelArco33 decide no 59:59 para seguir facendo ✨HISTORIA✨\n\n😏 Segundo ano consecutivo na F8 da #CopadelRey 🏆 🥳 \n\n🔜 #Santan...,0
5,🔴 GPT-4 de OpenAI - Primeras impresiones... 🔥 https://t.co/QEoIaw4pPA #PayPay配布 #PayPal #zelle #bizumods #Venmo #cashapp #bbva...,0
6,#Repost @gina.bancaria with @let.repost \n• • • • • •\nBora trabaia\n.\n.\n.\n.\n\n.\n.\n\n#bancario \n#bancaria \n#febraban \...,0
7,Uma das agências do Santander vai fechar no dia 15 de junho e sua localização pode surpreender. Confira o comunicado do banco!...,0
8,Preparado pra prova e para as outras fases que vierem. Vamo dale!\n#cnb #streamerup #streamerbr #twitchbr #gamerbr #cnbacademy...,0
9,🏦 #Santander (SANB11) sobe juros do crédito imobiliário pela segunda vez no ano\n#mercadofinanceiro #investimentos\nhttps://t....,0


In [0]:
pos_tweets = [ tweet for index, tweet in enumerate(data['texto_twitter']) if data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['texto_twitter']) if data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['texto_twitter']) if data['SA'][index] < 0]
# Printando as porcentagens:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['texto_twitter'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['texto_twitter'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(data['texto_twitter'])))

Percentage of positive tweets: 15.384615384615385%
Percentage of neutral tweets: 84.61538461538461%
Percentage de negative tweets: 0.0%


In [0]:
# Grava os resultados
#rdd_save = '/FileStore/tables/datatraining/r'+time.strftime("%I,%M%S")
resultados_rdd = df
df.to_parquet('twitter-analyse.parquet')

#df.to_csv("your_name.csv", columns = ['texto_twitter', 'SA'])

dbutils.fs.put("/FileStore/tables/datatraining/twitter-bbb.parquet", resultados_rdd)

#resultados_rdd.write.format("csv").save("/FileStore/tables/datatraining/twitter-bbb2.csv")
#dbutils.fs.put("/FileStore/my-stuff/my-file.txt", "This is the actual text that will be saved to disk. Like a 'Hello world!' example")

#resultados_rdd.saveAsTextFile(rdd_save)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-2919296939286047> in <cell line: 5>()
      3 
      4 resultados_rdd = df
----> 5 df.write.format("parquet").save("/FileStore/tables/datatraining/twitter-analyse.parquet")
      6 # df.to_parquet('twitter-analyse.parquet')
      7 

/databricks/python/lib/python3.9/site-packages/pandas/core/generic.py in __getattr__(self, name)
   5485         ):
   5486             return self[name]
-> 5487         return object.__getattribute__(self, name)
   5488 
   5489     def __setattr__(self, name: str, value) -> None:

AttributeError: 'DataFrame' object has no attribute 'write'